In [1]:
# !wget https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/malaysia.geojson

In [2]:
# !pip3 install bokeh distributed dask

In [3]:
import json
import geojson
import requests
import re
import numpy as np
import dask.array as da
import dask.dataframe as dd
from shapely.geometry import mapping, shape, MultiPoint

In [4]:
from dask.distributed import Client, progress
client = Client()
client

Client Scheduler: tcp://127.0.0.1:57557 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 16.00 GiB


In [6]:
with open('malaysia.geojson') as fopen:
    malaysia = json.load(fopen)

In [7]:
negeri = {k['properties']['name']:k['geometry'] for k in malaysia['features']}
negeri.keys()

dict_keys(['Federal Territory of Kuala Lumpur', 'Perlis', 'Sabah', 'Federal Territory of Putrajaya', 'Kedah', 'Sarawak', 'Penang', 'Johor', 'Kelantan', 'Melaka', 'Negeri Sembilan', 'Pahang', 'Perak', 'Selangor', 'Terengganu'])

In [8]:
def check(df):
    points = MultiPoint(df.values)
    try:
        ls = list(points.intersection(polygon))
        return [(l.x, l.y) for l in ls]
    except:
        return []

In [ ]:
!mkdir data

In [10]:
resolution = 0.005
for k in negeri:
    print(k)
    s = json.dumps(negeri[k])
    g1 = geojson.loads(s)
    polygon = shape(g1)
    latmin, lonmin, latmax, lonmax = polygon.bounds 
    lat_arange = da.arange(latmin, latmax, resolution)
    lon_arange = da.arange(lonmin, lonmax, resolution)
    x, y = da.meshgrid(lat_arange, lon_arange)
    x = da.round(x, 6)
    y = da.round(y, 6)
    x = da.reshape(x, (-1, 1))
    y = da.reshape(y, (-1, 1))
    concated = da.concatenate([x, y], axis = 1)
    df = dd.from_array(concated)
    df = df.repartition(npartitions = 100)
    check_partitions = df.map_partitions(check, meta=object)
    check_partitions = check_partitions.compute()
    inside = []
    for p in check_partitions:
        inside.extend(p)

    print(k, len(inside))
    with open(f'data/{k}-points.json', 'w') as fopen:
        json.dump(inside, fopen)

Federal Territory of Kuala Lumpur
Federal Territory of Kuala Lumpur 767
Perlis
Perlis 2668
Sabah
Sabah 240019
Federal Territory of Putrajaya
Federal Territory of Putrajaya 131
Kedah
Kedah 29459
Sarawak
Sarawak 404284
Penang
Penang 3364
Johor
Johor 61615
Kelantan
Kelantan 49052
Melaka
Melaka 5422
Negeri Sembilan
Negeri Sembilan 21704
Pahang
Pahang 116876
Perak
Perak 68097
Selangor
Selangor 25428
Terengganu
Terengganu 42032
